<a href="https://colab.research.google.com/github/vishal894/text_generation_model-NLP-/blob/main/text_generation_using_LSTM_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import string
import requests

In [2]:
response=requests.get('https://raw.githubusercontent.com/laxmimerit/poetry-data/master/adele.txt')

In [104]:
#response.text

In [4]:
data=response.text.split('\n')

In [5]:
data[0]

'Looking for some education'

In [6]:
data[8]

"If that's love in your eyes"

In [7]:
data=" ".join(data)

In [9]:
## Remove punctuation from data

In [10]:
import re

In [22]:
def data_clean(data):
  tokens=data.split()
  #tokens=str.maketrans('','',string.punctuation)
  punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
  #if tokens in punc:
    #tokens = tokens.replace(tokens, "")    
  #tokens=[w.translate(table) for w in tokens]
  tokens=[word for word in tokens if word.isalpha()]
  tokens=[word.lower() for word in tokens]
  return tokens


In [23]:
tokens=data_clean(data)

In [24]:
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
for token in tokens:
  if token in punc:
    tokens=tokens.replace(token,"")


In [25]:
len(tokens)

16598

In [26]:
vec_dim=len(set(tokens))

In [27]:
max_length=20+1
lines=[]
for i in range(max_length,len(tokens)):
  seq=tokens[i-max_length:i]
  line=' '.join(seq)
  lines.append(line)
  if i>2000:
    break;


In [28]:
print(len(lines))

1981


In [29]:
lines[0]

'looking for some education made my way into the night all that bullshit conversation you read the i bore you with'

In [82]:
import  numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [33]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(lines)

In [36]:
tokenizer.word_index


{'a': 19,
 'about': 205,
 'absence': 229,
 'affirmation': 222,
 'after': 305,
 'all': 16,
 'almost': 202,
 'already': 216,
 'always': 120,
 'and': 6,
 'anymore': 58,
 'apart': 153,
 'are': 27,
 'arms': 363,
 'as': 69,
 'ask': 154,
 'asking': 349,
 'at': 88,
 'babies': 227,
 'baby': 28,
 'back': 184,
 'bad': 170,
 'bare': 197,
 'be': 30,
 'beat': 138,
 'beating': 289,
 'been': 333,
 'before': 207,
 'best': 110,
 'better': 372,
 'between': 322,
 'bittersweet': 272,
 'blessings': 295,
 'blue': 180,
 'bore': 221,
 'born': 259,
 'both': 330,
 'bound': 263,
 'bread': 177,
 'breaking': 150,
 'breathless': 203,
 'bringing': 194,
 'bullshit': 385,
 'burn': 124,
 'but': 12,
 'by': 38,
 'california': 314,
 'call': 145,
 'called': 142,
 'calling': 234,
 'came': 249,
 'can': 22,
 'cares': 268,
 'caught': 379,
 'cause': 373,
 'claim': 352,
 'clear': 275,
 'clearly': 151,
 'close': 368,
 'clothes': 240,
 'coming': 337,
 'conversation': 384,
 'could': 40,
 'count': 294,
 'cried': 374,
 'cruel': 347,
 

In [37]:
embedded_vector=tokenizer.texts_to_sequences(lines)

In [74]:
#embedded_vector[0]

In [80]:
datalist = []
for d in embedded_vector:
  if len(d)>1:
    for i in range(2, len(d)):
      datalist.append(d[:i])
      #print(d[:i])

In [105]:
max_length = 20
sequences = pad_sequences(datalist, maxlen=max_length, padding='pre')
#sequences

In [39]:
embedded_vector=np.array(embedded_vector)

In [40]:
embedded_vector[0]

array([ 96,  20,  95, 386, 123,  11,  44,  93,   3,  92,  16,   9, 385,
       384,   2, 383,   3,   1, 221,   2,  47])

In [85]:
#X,y=embedded_vector[:,:-1],embedded_vector[:,-1]
X = sequences[:, :-1]
y = sequences[:, -1]

In [86]:
vocab_size=len(tokenizer.word_index)+1

In [87]:
len(tokenizer.word_index)

386

In [88]:
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [89]:
X.shape[1]

19

In [90]:
model=Sequential()
model.add(Embedding(vocab_size,20,input_length=seq_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))


In [103]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 19, 20)            7740      
_________________________________________________________________
lstm_12 (LSTM)               (None, 19, 100)           48400     
_________________________________________________________________
lstm_13 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_13 (Dense)             (None, 387)               39087     
Total params: 185,727
Trainable params: 185,727
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [92]:
model.fit(X,y,batch_size=32,epochs=120)

Epoch 1/120
1177/1177 [==============================] - 8s 6ms/step - loss: 4.6145 - accuracy: 0.0835
Epoch 2/120
1177/1177 [==============================] - 7s 6ms/step - loss: 3.1091 - accuracy: 0.2557
Epoch 3/120
1177/1177 [==============================] - 7s 6ms/step - loss: 2.1409 - accuracy: 0.4594
Epoch 4/120
1177/1177 [==============================] - 7s 6ms/step - loss: 1.5149 - accuracy: 0.6198
Epoch 5/120
1177/1177 [==============================] - 7s 6ms/step - loss: 1.0949 - accuracy: 0.7322
Epoch 6/120
1177/1177 [==============================] - 7s 6ms/step - loss: 0.8284 - accuracy: 0.7974
Epoch 7/120
1177/1177 [==============================] - 7s 6ms/step - loss: 1.2480 - accuracy: 0.7048
Epoch 8/120
1177/1177 [==============================] - 7s 6ms/step - loss: 1.5971 - accuracy: 0.5884
Epoch 9/120
1177/1177 [==============================] - 7s 6ms/step - loss: 0.8256 - accuracy: 0.7975
Epoch 10/120
1177/1177 [==============================] - 7s 6ms/step - l

In [97]:
poetry_length = 10
def generate_poetry(seed_text, n_lines):
  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = tokenizer.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

      y_pred = np.argmax(model.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in tokenizer.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    print(text)

In [98]:
seed_text = 'i love you'
generate_poetry(seed_text, 5)

treat your girl that i can give you all the
beat are gonna rolling in the i have no story
to make love to you love to ooh baby love
to yeah and i can tell by the way you
never had met rolling in the deep are gonna rolling


In [99]:
seed_text = 'i have no story'
generate_poetry(seed_text, 5)

to be told but heard one on you and gonna
wish you never had met rolling in the deep are
gonna rolling in the i have no story to be
home hello from the outside at least i can say
that maybe you could help me ease my mind i


In [102]:
loss, accuracy = model.evaluate(X, y, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 93.881345
